生成数据集

In [20]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l
def synthetic_data(w, b, num_examples):  #@save
    """生成X和y，其中的关系为y=Xw+b+噪声，w=[2,-3.4]^T"""
    X = torch.normal(0, 1, (num_examples, len(w)))
    # 产生一个服从N(0,1)，大小为num_examples X len(w)的正态分布矩阵
    # 这里服从正态分布的意思是每一个数是从正态分布中等可能取出的，而不是整个矩阵呈现正态分布的样子
    y = torch.matmul(X, w) + b
    # 计算矩阵向量积Xw，然后加上b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))
    # 最后的y是一个列向量，X是一个1000 x 2的矩阵

true_w = torch.tensor([2, -3.4]) # 生成基本数据结构：张量
true_b = 4.2
# 实际的w和b如上，也就是y=X[2, -3.4] + 4.2
features, labels = synthetic_data(true_w, true_b, 1000)
# 生成特征和标签
print('features:', features[0],'\nlabel:', labels[0])

features: tensor([-0.7881,  2.5178]) 
label: tensor([-5.9475])


读取数据集

In [21]:
def loadDataSets(data_arrays, batch_size, is_train=True):
    print(type(data_arrays))
    ds = data.TensorDataset(*data_arrays) #星号表示解开list写入其中参数。
    return data.DataLoader(dataset=ds, batch_size=batch_size, shuffle=is_train)
    # 这里的DataLoader是pytorch里面的可迭代对象。
    # 一般的pytorch的数据处理都是使用这三步：
    # 1.构造Dataset对象  2.通过DataLoader来构造迭代对象  3.逐步迭代数据

batch_size = 4
data_iter = loadDataSets((features, labels), batch_size)
next(iter(data_iter))

<class 'tuple'>


[tensor([[-1.5064, -0.6793],
         [ 0.2044, -1.5074],
         [-0.1635,  0.3499],
         [ 0.5409, -1.7088]]),
 tensor([[ 3.4933],
         [ 9.7160],
         [ 2.6644],
         [11.1012]])]

定义模型

In [22]:
# nn是神经网络的缩写
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))
# 这里的nn.Sequential是一个层容器，里面的参数就是层数。
# 这里的nn.Linear(2, 1)就是一个线性层，一个2对1的线性层，本质上就是Y=ax1+bx2
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)
# 这里是指定每个权重参数应该从均值为0、标准差为0.01的正态分布中随机采样，偏置参数将初始化为零。
# 上面的net[0]是访问了Sequential层容器中的第一层模型的参数，也就是线性模型的参数
# net[0].weight.data是访问了权重参数，也就对应Linear里面的那个2
# net[0].weight.data.normal_(0, 0.01)则是规定权重参数从均值为0、标准差为0.01的正态分布中随机采样
# net[0].bias.data.fill_(0)则是将偏置放为0

tensor([0.])

定义损失函数

In [23]:
loss = nn.MSELoss()
# 定义均方误差

定义优化算法

In [24]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)
# 定义了小批量随机梯度下降算法，其中net.parameters()定义了要优化的参数，lr=0.03是学习率超参数

训练

In [25]:
num_epochs = 100
for epoch in range(num_epochs):
    for X, y in data_iter:
        # 每一次迭代data_iter都可以得到一部分样本，这个挺有意思
        l = loss(net(X) ,y)
        # 计算对于X这一个小样本的误差，后面只对这个小样本进行梯度计算
        trainer.zero_grad()
        # 梯度清零，否则后续训练梯度会累积
        l.backward()
        # 反向传播梯度
        # 似乎得先反向传播了梯度后才可以更新模型的参数
        trainer.step()
        # 更新模型的参数
    l = loss(net(features), labels)
    # 计算误差
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000105
epoch 2, loss 0.000107
epoch 3, loss 0.000104
epoch 4, loss 0.000105
epoch 5, loss 0.000107
epoch 6, loss 0.000108
epoch 7, loss 0.000105
epoch 8, loss 0.000109
epoch 9, loss 0.000105
epoch 10, loss 0.000109
epoch 11, loss 0.000107
epoch 12, loss 0.000110
epoch 13, loss 0.000107
epoch 14, loss 0.000106
epoch 15, loss 0.000104
epoch 16, loss 0.000105
epoch 17, loss 0.000104
epoch 18, loss 0.000107
epoch 19, loss 0.000108
epoch 20, loss 0.000104
epoch 21, loss 0.000104
epoch 22, loss 0.000105
epoch 23, loss 0.000108
epoch 24, loss 0.000105
epoch 25, loss 0.000104
epoch 26, loss 0.000106
epoch 27, loss 0.000109
epoch 28, loss 0.000105
epoch 29, loss 0.000107
epoch 30, loss 0.000106
epoch 31, loss 0.000109
epoch 32, loss 0.000110
epoch 33, loss 0.000106
epoch 34, loss 0.000108
epoch 35, loss 0.000105
epoch 36, loss 0.000105
epoch 37, loss 0.000104
epoch 38, loss 0.000106
epoch 39, loss 0.000106
epoch 40, loss 0.000106
epoch 41, loss 0.000108
epoch 42, loss 0.000108
e

In [26]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([0.0008, 0.0010])
b的估计误差： tensor([-3.2425e-05])
